![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/research/overview
#load "../QuantConnect.csx"    
using QuantConnect.Data.UniverseSelection;

var qb = new QuantBook();
var symbols = LiquidETFUniverse.SP500Sectors;

var history = qb.History(symbols, 360, Resolution.Daily);

In [2]:
// Calculates the % returns of our symbols over the historical data period
var returns = new Dictionary<Symbol, List<decimal>>();
var last = new Dictionary<Symbol, decimal>();
foreach(var slice in history){
    foreach(var symbol in slice.Bars.Keys){
        if(!returns.ContainsKey(symbol)){
            returns.Add(symbol, new List<decimal>());
            last.Add(symbol, (decimal)slice.Bars[symbol].Close);
        }
        var change = (decimal) ((decimal)slice.Bars[symbol].Close - last[symbol])/last[symbol];
        returns[symbol].Add(change);
    }    
}



var means = new Dictionary<Symbol, decimal>();

// Calculate mean price over 30 day period for each symbol
foreach(var symbol in returns.Keys){
    if(!means.ContainsKey(symbol)){
        var avg = returns[symbol].Average();
        means.Add(symbol, avg);
    }
}

var std = new Dictionary<Symbol, decimal>();

// Calculate standard deviation of prices for each symbol
foreach(var symbol in returns.Keys){
    if(!std.ContainsKey(symbol)){
        var average = means[symbol];
        var sumOfSquaresOfDifferences = returns[symbol].Select(val => (val - average) * (val - average)).Sum();
        var sd = (Decimal)Math.Sqrt((double) (sumOfSquaresOfDifferences / returns[symbol].Count()));

        std.Add(symbol, sd);
    }
}


In [3]:
var ZScores = new Dictionary<Symbol, List<decimal>>();

// Calculate the Z-Score for each return in the time series
foreach(var symbol in returns.Keys){
    
    if(!ZScores.ContainsKey(symbol)){
        ZScores.Add(symbol, new List<decimal>());
    }
    
    foreach(var ret in returns[symbol]){
        var score = (decimal)((ret - means[symbol])/std[symbol]);
        ZScores[symbol].Add(score);    
    }
}

foreach(var symbol in ZScores.Keys){
    Console.WriteLine(symbol + ", " + ZScores[symbol].Count);
}